In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from PIL import Image

2023-06-13 18:23:31.837997: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
PATH_CSV_LABELS = 'tabellaPseudoLabels.csv'
PATH_DATASET_LAVORATO = './Dati Sensori/Dataset Sensori Lavorato'
PATH_MODELLO = 'Modelli/modello_pesi_inizializzati_v1'

N_IMG = 1
FORMATO_IMG = 'jpg'
HEIGHT = 512
WIDTH = 512
TARGET_SIZE = (WIDTH, HEIGHT)

In [3]:
df_labels = pd.read_csv(PATH_CSV_LABELS)
df_labels.head(5)

,Acquisizione,Label,Label_norm
0,12_M_1,0.558931,14.387763
1,12_M_2,0.848155,90.057848
2,12_M_3,0.886156,100.000000
3,12_M_4,0.771945,70.119024
4,12_M_5,0.744003,62.808388


In [4]:
def get_id_soggetto(id_acquisizione):
    id_acquisizione = id_acquisizione.split('_')
    n_soggetto = id_acquisizione[0]
    sesso_soggetto = id_acquisizione[1]
    return f"{n_soggetto}_{sesso_soggetto}"

In [5]:
X = []
y = []

for id_acquisizione in df_labels.Acquisizione:
    nome_file = f"{id_acquisizione}_img_{N_IMG}.{FORMATO_IMG}"
    path_img = os.path.join(PATH_DATASET_LAVORATO, get_id_soggetto(id_acquisizione), id_acquisizione, nome_file)
    image = Image.open(path_img)
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    
    pseudo_label = df_labels[df_labels.Acquisizione == id_acquisizione].Label_norm
    
    X.append(normalized_image_array)
    y.append(pseudo_label)
    
X = np.array(X)
y = np.array(y)

In [6]:
model = keras.models.load_model(PATH_MODELLO)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 255, 255, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2080800)           0         
                                                                 
 dense (Dense)               (None, 64)                133171264 
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 133,171,649
Trainable params: 133,171,649
Non-trainable params: 0
__________________________________________

In [7]:
model.fit(x=X, y=y, batch_size=1, epochs=20)

Epoch 1/20
30/30 [==============================] - 15s 493ms/step - loss: 25.2234 - mae: 25.2234
Epoch 2/20
30/30 [==============================] - 14s 483ms/step - loss: 24.2245 - mae: 24.2245
Epoch 3/20
30/30 [==============================] - 15s 483ms/step - loss: 22.6327 - mae: 22.6327
Epoch 4/20
30/30 [==============================] - 14s 483ms/step - loss: 22.1368 - mae: 22.1368
Epoch 5/20
30/30 [==============================] - 15s 485ms/step - loss: 20.9673 - mae: 20.9673
Epoch 6/20
30/30 [==============================] - 15s 489ms/step - loss: 19.6179 - mae: 19.6179
Epoch 7/20
30/30 [==============================] - 15s 486ms/step - loss: 18.9098 - mae: 18.9098
Epoch 8/20
30/30 [==============================] - 15s 487ms/step - loss: 17.3051 - mae: 17.3051
Epoch 9/20
30/30 [==============================] - 15s 490ms/step - loss: 15.5713 - mae: 15.5713
Epoch 10/20
30/30 [==============================] - 15s 484ms/step - loss: 13.1616 - mae: 13.1616
Epoch 11/20
30/30 [

In [8]:
model.predict(X)

1/1 [==============================] - 1s 503ms/step


array([[ 19.064325],
       [ 89.98518 ],
       [101.65917 ],
       [ 73.417816],
       [ 67.47574 ],
       [ 10.503828],
       [ 80.16316 ],
       [ 55.02753 ],
       [ 80.8325  ],
       [ 75.33259 ],
       [ 36.09754 ],
       [ 65.48462 ],
       [ 75.72285 ],
       [ 12.284901],
       [ 60.757412],
       [ 39.196198],
       [ 39.354538],
       [ 68.43273 ],
       [ 71.47165 ],
       [ 68.38305 ],
       [ 36.6414  ],
       [ 48.771973],
       [ 52.153706],
       [ 61.76799 ],
       [ 78.03122 ],
       [ 25.86473 ],
       [ 25.844954],
       [ 14.370479],
       [ 43.98975 ],
       [ 28.897617]], dtype=float32)

In [9]:
y

array([[ 14.38776324],
       [ 90.05784833],
       [100.        ],
       [ 70.11902421],
       [ 62.8083878 ],
       [  0.        ],
       [ 75.6853248 ],
       [ 53.78656739],
       [ 73.88803792],
       [ 69.96671962],
       [ 31.43078081],
       [ 61.84041709],
       [ 73.70590257],
       [  1.90248829],
       [ 61.10732865],
       [ 31.94502288],
       [ 36.98859557],
       [ 64.10041713],
       [ 71.95538621],
       [ 62.96287593],
       [ 33.46659409],
       [ 45.18621211],
       [ 48.7416397 ],
       [ 54.43098089],
       [ 75.04505273],
       [ 21.64331626],
       [ 25.37730258],
       [  6.16541405],
       [ 41.17895281],
       [ 24.97404123]])